In [2]:
# Import of the main utilities
from nltk.wsd import lesk 
from nltk.corpus import stopwords, wordnet, semcor
from nltk import tree


In [5]:
def lesk2(context_sentence, ambiguous_word, pos=None):
    # Attempt to make lesk more accurate removing stop words
    stop_words = set(stopwords.words('english'))
    context = set(context_sentence)
    synsets = wordnet.synsets(ambiguous_word)

    if pos:
        synsets = [ss for ss in synsets if str(ss.pos()) == pos]
    if not synsets:
        return None
    _, sense = max(
        (len(
            context.intersection(set(ss.definition().split())-stop_words)), ss
        ) for ss in synsets
    )
    return sense

In [52]:
def retrieve_sent(ambiguos_word, all_tagged_sents):
    # This method retrieve all the sentences containing the specified ambiguos word
    # and the corresponding synset from a tagged corpora structured as list of sentences. 
    # Each sentence is represented as a list of tagged chunks (in tree form).
    saved_tagged_sent = []
    saved_synset = []
    
    for tagged_sent in all_tagged_sents[0:1000]:
        sent_to_save = False
        sent_words = []
        for tagged_chunk in tagged_sent:
            synset = ""
            if isinstance(tagged_chunk, tree.Tree):
                current_word = tagged_chunk.leaves()
                sent_words = sent_words + current_word
                synset = tagged_chunk.label()
            else:
                current_word = tagged_chunk
                sent_words = sent_words + current_word
                if len(tagged_chunk) == 2:
                    synset = tagged_chunk[1]
            if ambiguos_word in current_word:
                sent_to_save = True
                saved_synset.append(synset)
        if sent_to_save:
            saved_tagged_sent.append(sent_words)
    return (saved_tagged_sent, saved_synset)

[['If', 'the', 'Orioles', 'are', 'to', 'break', 'their', 'losing', 'streak', 'within', 'the', 'next', 'two', 'days', ',', 'it', 'will', 'have', 'to', 'be', 'at', 'the', 'expense', 'of', 'the', 'American', 'League', 'champion', 'New', 'York', 'Yankees', ',', 'who', 'come', 'in', 'here', 'tomorrow', 'for', 'a', 'night', 'game', 'and', 'a', 'single', 'test', 'Sunday', 'afternoon', '.'], ['The', 'Texans', 'made', 'themselves', 'a', 'comforting', 'break', 'on', 'the', 'opening', 'kickoff', 'when', 'Denver', "'s", 'Al', 'Carmichael', 'was', 'jarred', 'loose', 'from', 'the', 'ball', 'when', 'Dave', 'Grayson', ',', 'the', 'speedy', 'halfback', ',', 'hit', 'him', 'and', 'Guard', 'Al', 'Reynolds', 'claimed', 'it', 'for', 'Dallas', '.'], ['The', 'opinion', 'continues', 'here', 'that', 'with', 'a', '162', '-', 'game', 'schedule', ',', 'pitching', 'spread', 'thin', 'through', 'a', '10', '-', 'team', 'league', 'and', 'a', 'most', 'inviting', 'target', 'in', 'Los', 'Angeles', "'", 'Wrigley', 'Field',

In [6]:
# Test retrieve_sentence method
all_tagged_sents = semcor.tagged_sents(tag='sem')
retrievedSent = retrieve_sent("break", all_tagged_sents)
print(retrievedSent)

Synset('fault.n.04')
Synset('rupture.n.02')
(geology) a crack in the earth's crust resulting from the displacement of one side with respect to the other
a personal or social separation (as between opposing factions)


In [22]:
# Check differences obtained by using regular lesk and lesk2
sent3 = "the break was short".split()

sense1=lesk(sent3, 'break', 'n')
sense2=lesk2(sent3, 'break', 'n')

print(sense1)
print(sense1.definition())

print(sense2)
print(sense2.definition())
# Which one is better???

['a', 'b', 'c', 'd']
